### Setup

In [36]:
import os
import openai
import numpy as np
from tqdm import tqdm
import time

# set up openai api key
api_key_location=os.path.expanduser(os.path.join("~/.ssh/", "openai")) 
with open(api_key_location) as f:
    openai.api_key = f.read().strip()

### Generate prompts (English)

In [31]:
# generate_chat_prompt will take as input a tuple of indices corresponding to concepts in `labels`
def generate_prompt_eng(color):
    prompt = f"""Please name the following color using one word only.\n"""
    prompt += f'''Color 1: {color}\n'''
    return prompt

# try an example
color="#000000"
prompt=generate_prompt_eng(color)
print(prompt)
messages=[{"role":"user", "content":prompt}]
print(messages)

Please name the following color using one word only.
Color 1: #000000

[{'role': 'user', 'content': 'Please name the following color using one word only.\nColor 1: #000000\n'}]


### Generate prompts (Japanese)

In [41]:
# generate_chat_prompt will take as input a tuple of indices corresponding to concepts in `labels`
def generate_prompt_jp(color):
    prompt = f"""この色を一言で表現してください。\n"""
    prompt += f'''色 1: {color}\n'''
    return prompt

# try an example
color="#000000"
prompt=generate_prompt_jp(color)
print(prompt)
messages=[{"role":"user", "content":prompt}]
print(messages)

この色を一言で表現してください。
色 1: #000000

[{'role': 'user', 'content': 'この色を一言で表現してください。\n色 1: #000000\n'}]


### Configure GPT

In [32]:
temperature = 0.7 # default temperature setting
model = "gpt-3.5-turbo"
seed = 1

### Collect GPT Responses


In [37]:
def openai_chat_completion(messages, model, temperature):
    completion=False
    i=0
    while not completion:
        i+=1
        try:
            completion = openai.chat.completions.create(
                model=model,
                messages=messages,
                temperature=temperature,
            )
        except Exception as e:
            if i>=5: # if error persis after 5 attempts, give up
                return False, False
            if i>=4:
                print(f'Attempt {i} failed: {e}')
            elif i>=3: print(f'Attempt {i} failed.')
            time.sleep(5) # wait before pinging the API again in case error is due to rate limit
    choices = [dict(choice.items()) for choice in completion.choices]
    return choices, completion.created
    # return response

In [26]:
# get list of colors from txt file
file = open('colors.txt', 'r')
colors = file.read().splitlines()
file.close()

#### Run English Prompts

In [39]:
responses = []
for i, color in enumerate(tqdm(colors)):
    prompt = generate_prompt_eng(color)
    response=False
    answer=False
    attempt=0
    while not answer:
        attempt += 1
        messages=[{'role':'user', 'content':prompt}]
        choices, created=openai_chat_completion(messages, model, temperature)

        try:
            answer = choices[0].message.content
            responses.append(answer)
            
        # if we fail parse out an answer 10 times in a row, skip entry
        except Exception as e: 
            print(answer)
            print(e)
            answer=False
            if attempt>10:
                print('Error', e)

#### Run Japanese Prompts

In [ ]:
responses = []
for i, color in enumerate(tqdm(colors)):
    prompt = generate_prompt_jp(color)
    response=False
    answer=False
    attempt=0
    while not answer:
        attempt += 1
        messages=[{'role':'user', 'content':prompt}]
        choices, created=openai_chat_completion(messages, model, temperature)

        try:
            answer = choices[0].message.content
            responses.append(answer)
            
        # if we fail parse out an answer 10 times in a row, skip entry
        except Exception as e: 
            print(answer)
            print(e)
            answer=False
            if attempt>10:
                print('Error', e)

### Analyze the data